In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql import functions as func
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, LongType
import codecs
spark = SparkSession.builder.appName('heros').getOrCreate()

In [3]:
schema = StructType([StructField('id', IntegerType(), True),
                    StructField('name', StringType(), True)])

In [4]:
names = spark.read.schema(schema).option('sep', ' ').csv('data/Marvel+Names')

In [5]:
lines = spark.read.text("data/Marvel+Graph")

In [6]:
lines.show()

+--------------------+
|               value|
+--------------------+
|5988 748 1722 375...|
|5989 4080 4264 44...|
|5982 217 595 1194...|
|5983 1165 3836 43...|
|5980 2731 3712 15...|
|5981 3569 5353 40...|
|5986 2658 3712 26...|
|5987 2614 5716 17...|
|5984 590 4898 745...|
|5985 3233 2254 21...|
|6294 4898 1127 32...|
|270 2658 3003 380...|
|271 4935 5716 430...|
|272 2717 4363 408...|
|273 1165 5013 511...|
|274 3920 5310 402...|
|275 4366 3373 158...|
|276 2277 5251 480...|
|277 1068 3495 619...|
|278 1145 667 2650...|
+--------------------+
only showing top 20 rows



In [9]:
connections = lines.withColumn('id', func.split(func.col("value"), " ")[0]).withColumn("connections", func.size(func.split(func.col("value"), " "))-1).groupBy('id').agg(func.sum("connections").alias("connections"))

In [10]:
connections.show()

+----+-----------+
|  id|connections|
+----+-----------+
| 691|          7|
|1159|         12|
|3959|        143|
|1572|         36|
|2294|         15|
|1090|          5|
|3606|        172|
|3414|          8|
| 296|         18|
|4821|         17|
|2162|         42|
|1436|         10|
|1512|         12|
|6194|         15|
|6240|         12|
| 829|         38|
|2136|          7|
|5645|         21|
|2069|        264|
| 467|          1|
+----+-----------+
only showing top 20 rows



In [11]:
mostPopular = connections.sort(func.col("connections").desc()).first()

In [12]:
mostPopularNmae = names.filter(func.col("id") == mostPopular[0]).select('name').first()

In [15]:
print(mostPopularNmae[0] + " is the most popular superhero with " + str(mostPopular[1]) + " co-apperances")

CAPTAIN AMERICA is the most popular superhero with 1937 co-apperances
